In [42]:
import pandas as pd
from glob import glob
import networkx as nx
import numpy as np

In [43]:
para = pd.read_csv('../../../../../../../../../../../Volumes/sandisk1/datasets/splicing/Athma_cryptic_splicing/GRCh38_ensembl95.Paralogs.csv')
para


,Unnamed: 0,Gene Internal ID,Gene stable ID,Gene Synbol,HasParalog,Paralog stable ID,Paralog Synbol
0,0,319832,ENSG00000284532,MIR4723,False,NaN,NaN
1,1,319833,ENSG00000238933,RF00019,True,ENSG00000202361,RF00019
2,2,319833,ENSG00000238933,RF00019,True,ENSG00000201843,RF00019
3,3,319833,ENSG00000238933,RF00019,True,ENSG00000200008,RF00019
4,4,319833,ENSG00000238933,RF00019,True,ENSG00000201466,RF00019
...,...,...,...,...,...,...,...
3618911,3618911,324201,ENSG00000285013,AC243837.3,False,NaN,NaN
3618912,3618912,324202,ENSG00000285154,AC243837.4,False,NaN,NaN
3618913,3618913,324203,ENSG00000284933,ARHGEF5,False,NaN,NaN
3618914,3618914,324204,ENSG00000285363,MTRF1LP2,False,NaN,NaN


In [44]:
gene_data = pd.read_csv('../../../../../../../../../../../Volumes/sandisk1/datasets/splicing/Athma_cryptic_splicing/GRCh38_ensembl95.genes.csv')

gene_data = gene_data.rename(columns={'Gene Chromosome':'Chromosome/scaffold name', 'Gene Strand':'Strand','Gene Synbol':'Gene name'})
gene_data = gene_data[['Gene stable ID','Chromosome/scaffold name','Gene start (bp)','Gene end (bp)','Strand','Gene name','Gene Type']].set_index('Gene stable ID')
print(gene_data.index.has_duplicates)
gene_data.to_csv('../../../../../../../../../../../Volumes/sandisk1/datasets/splicing/protein_coding/gene_data.csv')
gene_data

False


,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Strand,Gene name,Gene Type
Gene stable ID,,,,,,
ENSG00000284532,17,28360654,28360734,1,MIR4723,miRNA
ENSG00000238933,9,61642383,61642494,1,RF00019,misc_RNA
ENSG00000275693,10,88992370,88992539,1,RF02116,misc_RNA
ENSG00000275451,15,62343029,62343138,1,MIR6085,miRNA
ENSG00000222870,7,94495299,94495404,1,RNU6-1328P,snRNA
...,...,...,...,...,...,...
ENSG00000285013,CHR_HSCHR7_3_CTG4_4,144396023,144396316,1,AC243837.3,protein_coding
ENSG00000285154,CHR_HSCHR7_3_CTG4_4,144414782,144415702,1,AC243837.4,protein_coding
ENSG00000284933,CHR_HSCHR7_3_CTG4_4,144457568,144474951,1,ARHGEF5,protein_coding


In [45]:
only_observed_orthos = para[para['Gene stable ID'].isin(gene_data[gene_data['Gene Type'] == 'protein_coding'].index.unique())]

only_observed_orthos = only_observed_orthos[['Gene stable ID', 'Paralog stable ID']].dropna(how='any')

only_observed_orthos = only_observed_orthos.apply(lambda x: (x['Gene stable ID'],x['Paralog stable ID']),
                                                            axis=1)
only_observed_orthos

212478     (ENSG00000278198, ENSG00000273496)
212479     (ENSG00000278198, ENSG00000276256)
216903     (ENSG00000198763, ENSG00000198886)
216904     (ENSG00000198763, ENSG00000198786)
219472     (ENSG00000198886, ENSG00000198786)
                          ...                
3454448    (ENSG00000278782, ENSG00000270472)
3454449    (ENSG00000278782, ENSG00000271130)
3454450    (ENSG00000278782, ENSG00000259680)
3454451    (ENSG00000278782, ENSG00000211957)
3454452    (ENSG00000278782, ENSG00000211944)
Length: 274882, dtype: object

In [46]:
gene_data[gene_data['Gene Type'] == 'protein_coding'].apply(lambda x: (x.name, {'chr':x['Chromosome/scaffold name'], 'strand':x['Strand']}), axis=1)

Gene stable ID
ENSG00000275568    (ENSG00000275568, {'chr': 'CHR_HSCHR15_4_CTG8'...
ENSG00000275049    (ENSG00000275049, {'chr': 'CHR_HSCHR15_5_CTG8'...
ENSG00000276804    (ENSG00000276804, {'chr': 'CHR_HSCHR19LRC_LRC_...
ENSG00000276565    (ENSG00000276565, {'chr': 'CHR_HSCHR11_1_CTG6'...
ENSG00000281024    (ENSG00000281024, {'chr': 'CHR_HG2030_PATCH', ...
                                         ...                        
ENSG00000284827    (ENSG00000284827, {'chr': 'CHR_HSCHR7_3_CTG4_4...
ENSG00000285013    (ENSG00000285013, {'chr': 'CHR_HSCHR7_3_CTG4_4...
ENSG00000285154    (ENSG00000285154, {'chr': 'CHR_HSCHR7_3_CTG4_4...
ENSG00000284933    (ENSG00000284933, {'chr': 'CHR_HSCHR7_3_CTG4_4...
ENSG00000285114    (ENSG00000285114, {'chr': 'CHR_HSCHR8_7_CTG7',...
Length: 22686, dtype: object

In [47]:
G =  nx.Graph()
G.add_nodes_from(gene_data[gene_data['Gene Type'] == 'protein_coding'].apply(lambda x: (x.name, {'chr':x['Chromosome/scaffold name'], 'strand':x['Strand']}), axis=1))
G.add_edges_from(only_observed_orthos)
print(f'{G.number_of_nodes()=}')

all_nodes = pd.Series(G.nodes).index
print('These nodes are missing from the paralog search')
print(all_nodes[~all_nodes.isin(para['Gene stable ID'].unique())])

ortho_groups = list(nx.connected_components(G))
print(f'captured {len(ortho_groups)} ortho groups')

test_set = [
    (G.degree[node], node)
    for node, data in G.nodes(data=True) 
    if data.get('chr') in ['1', '3', '5', '7', '9'] and G.degree[node] == 0
]

neghbors, test_nodes = zip(*test_set)
print(np.unique(neghbors))
print(len(test_nodes))

G.remove_nodes_from(test_nodes)


ortho_groups = list(nx.connected_components(G))
len(ortho_groups)

rng = np.random.default_rng(42)

perm = rng.permutation(len(ortho_groups))
chunks = np.array_split(perm, 5)
cvs = [[ortho_groups[i] for i in chunk] for chunk in chunks ]
cvs = [set.union(*cv) for cv in cvs]
cvs


G.number_of_nodes()=22889
These nodes are missing from the paralog search
Index([], dtype='object')
captured 9662 ortho groups
[0]
1148


[{'ENSG00000169255',
  'ENSG00000188086',
  'ENSG00000132294',
  'ENSG00000016490',
  'ENSG00000090861',
  'ENSG00000167182',
  'ENSG00000168758',
  'ENSG00000165819',
  'ENSG00000148136',
  'ENSG00000184924',
  'ENSG00000204842',
  'ENSG00000006607',
  'ENSG00000196391',
  'ENSG00000168769',
  'ENSG00000102290',
  'ENSG00000136982',
  'ENSG00000023228',
  'ENSG00000152092',
  'ENSG00000152782',
  'ENSG00000234443',
  'ENSG00000137936',
  'ENSG00000148942',
  'ENSG00000223952',
  'ENSG00000110851',
  'ENSG00000099331',
  'ENSG00000110665',
  'ENSG00000100084',
  'ENSG00000197403',
  'ENSG00000105366',
  'ENSG00000151320',
  'ENSG00000182013',
  'ENSG00000165972',
  'ENSG00000127831',
  'ENSG00000274636',
  'ENSG00000165863',
  'ENSG00000156261',
  'ENSG00000214654',
  'ENSG00000003989',
  'ENSG00000183647',
  'ENSG00000230995',
  'ENSG00000060762',
  'ENSG00000282530',
  'ENSG00000171103',
  'ENSG00000109689',
  'ENSG00000273111',
  'ENSG00000155249',
  'ENSG00000100092',
  'ENSG000001

In [48]:
gene_to_split = pd.DataFrame()
gene_to_split['gene'] = list(set.union(set(test_nodes), set.union(*cvs)))

gene_to_split['Test_set'] = False
gene_to_split.loc[gene_to_split['gene'].isin(test_nodes),'Test_set'] = True

for i, cv in enumerate(cvs):
    gene_to_split[f'CV{i}'] = False
    gene_to_split.loc[gene_to_split['gene'].isin(cv),f'CV{i}'] = True

gene_to_split.to_csv('../../../../../../../../../../../Volumes/sandisk1/datasets/splicing/protein_coding/gene_split.csv')
gene_to_split

,gene,Test_set,CV0,CV1,CV2,CV3,CV4
0,ENSG00000285542,False,False,False,True,False,False
1,ENSG00000132294,False,True,False,False,False,False
2,ENSG00000016490,False,True,False,False,False,False
3,ENSG00000167182,False,True,False,False,False,False
4,ENSG00000168758,False,True,False,False,False,False
...,...,...,...,...,...,...,...
22884,ENSG00000177666,False,False,False,False,True,False
22885,ENSG00000130227,False,True,False,False,False,False
22886,ENSG00000203811,False,False,True,False,False,False
22887,ENSG00000162298,False,False,False,True,False,False


In [49]:
gene_to_split.drop(columns='gene').sum()

Test_set    1148
CV0         4586
CV1         4495
CV2         4117
CV3         4252
CV4         4291
dtype: int64